# Object detectie


> We hebben nu de baselink-camera transformatie, we kunnen dus punten gezien vanuit de camera projecteren in baselink frame zodat de robot de positie van het object kent. In dit script zal je een foto van een object in het robot veld inladen waarna je hierop enkele image processing stappen uitvoert zodat je de positie van het object in pixelcoördinaten kan bepalen. Deze kan je met de intrinsieke matrix omzetten naar metrische coördinaten in camera frame dewelke je vervolgens met de baselink-camera transformatie kan omzetten in robot frame. 

# 1. Importeer bibliotheken

Deze bibliotheken zijn nodig voor het volbrengen van dit script. In de CameraCalibration bibliotheek (die je vindt in de Classes folder op je Google Drive) vind je de code achter de commando's die je hier zal gebruiken. 

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/object_pose_estimation_online')
from Classes.CameraCalibration import *
from google.colab.patches import cv2_imshow


# 2. Importeer alle beelden voor object detectie

> De afbeelding van het object in het robotveld vind je onder de map /content/drive/My Drive/object_pose_estimation_online/data/object_detection_images. Als je dit uitprint kan je zien dat er een padnaam aanwezig is naar een afbeelding met een ronde vorm. 



In [ ]:
# Get all image's path names
object_detection_images_file = '/content/drive/My Drive/object_pose_estimation_online/data/object_detection_images/*.png'
images_path_names = get_image_path_names(object_detection_images_file)
print(images_path_names)

> We kunnen terug kijken hoe een dergelijke afbeelding eruit ziet. 

In [ ]:
# Preview image
image_path = images_path_names[0]
image = cv2.imread(image_path)
cv2_imshow(image)

# 3. Image processing



> In de volgende stappen zal je enkele image processing stappen uitvoeren die het ruwe RGB beeld zullen verwerken naar de uiteindelijke positie van het object in pixelcoördinaten. We zetten het beeld eerst om naar grayscale om de grote hoeveelheid data te reduceren. Kleur informatie is in deze toepassing namelijk niet van belang. 



In [ ]:
# To grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2_imshow(gray_image)



> Vervolgens filteren we het beeld met een 7x7 Gaussiaanse filter om de hoogfrequente ruis eruit te krijgen. Dit zal ons helpen bij het detecteren van 'edges' in het beeld.



In [ ]:
# Gaussian blur
blurred_image = cv2.GaussianBlur(gray_image, (7, 7), 0)
cv2_imshow(blurred_image)



> Hier komt de stap waarbij we in de geblurde afbeelding randen zullen detecteren. Het gebruikte 'Canny Edge' algoritme zal pixels detecteren met een hoge intensiteitsgradiënt (groot verschil in pixelwaardes tussen twee opeenvolgende pixels). Als output krijgen we een binair (enkel 0 en 255 waarden) beeld met alle randen.



In [ ]:
# Canny Edge detection
canny = cv2.Canny(blurred_image, 50, 200)
cv2_imshow(canny)



> Je kan zien dat er links en rechts wat dingen mee gedetecteerd worden die we niet in ons resultaat willen. We zullen het beeld dus knippen. We definiëren met andere woorden een 'Region of Interest' (ROI) met enkel de ruimte die voor ons interessant is, namelijk het robot veld. We kijken eerst naar het aantal pixels in u- en v- richting dat de afbeelding heeft, en daarop baseren we ons om de pixels buiten de ROI op 0 te zetten (zwart).



In [ ]:
# Shape of image in u- and v- pixels
print("Image shape: " + str(canny.shape))

In [ ]:
# Define ROI
canny[:, 0:450] = 0  # All pixels from u-value 0 till 450 are put to zero for all v
canny[:, 900:1280] = 0  # All pixels from u-value 900 till 1280 are put to zero for all v
roi = canny
cv2_imshow(roi)



> Als volgt gaan we de gedetecteerde randen die slechts 1 pixel breed zijn 'dilaten' (zoek op) om de eventuele gaten die in de rand zouden voorkomen te dichten. 



In [ ]:
# Dilate image
roi_d = cv2.dilate(roi, None, iterations=2)
cv2_imshow(roi_d)



> Vervolgens 'eroderen' (zoek op) we de afbeelding terug om de rand weer te verkleinen. Maar de gaten blijven zo wel opgevuld zodat we zeker zijn van een continue rand. 



In [ ]:
# Erode image
roi_e = cv2.erode(roi_d, None, iterations=2)
cv2_imshow(roi_e)

# 4. Contour detection



> Nu we de image processing hebben beïndigd kunnen we overgaan tot het detecteren van 'contouren' (zoek op) in de afbeelding. Deze kunnen worden gevisualiseerd op de afbeelding. 



In [ ]:
# Find contours
contours = cv2.findContours(roi_e, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(contours)
image_with_contours = cv2.drawContours(image.copy(), contours, -1, (0, 255, 0), 3)
cv2_imshow(image_with_contours)



> We kunnen rond deze contour een bounding box definiëren. 



In [ ]:
# Compute the bounding box of the contour
box = cv2.minAreaRect(contours[0])
box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
image_with_box = cv2.drawContours(image.copy(), [box.astype("int")], -1, (0, 255, 0), 2)
cv2_imshow(image_with_box)



> Als we van de vier hoekpunten van deze bounding box gedefiniëerd in pixelcoördinaten de gemiddelde coördinaat nemen, dan bekomen we het centerpunt van ons object in pixelcoördinaten. 



In [ ]:
# Print box coördinates
print("Coördinaten van de bounding box in pixel coördinaten (u,v): \n\n" + str(box))
mean_coordinate = np.mean(box, axis=0)
print("\nCenterpunt van de box in pixel coördinaten (u,v): \n\n" + str(mean_coordinate))



> We kunnen dit visualiseren door een punt te projecteren op de afbeelding met deze pixel coördinaten. Dit punt zullen we nu projecteren in baselink frame.



In [ ]:
cv2.circle(image_with_box, tuple(mean_coordinate), 3, [255, 0, 0], thickness=3)
cv2_imshow(image_with_box)

# 5. Definiëren van de baselink-target transformatie

> We weten nu op welk coördinaat het object zich bevindt in pixel coördinaten. Dit dienen we nog om te zetten naar camera coördinaten en naar baselink coördinaten. Hiervoor laden we terug de eerder berekende intrinsieke matrix en de baselink-camera matrix.



In [ ]:
# Load intrinsic camera matrix and distortion coefficients
intrinsic_camera_matrix_file = '/content/drive/My Drive/object_pose_estimation_online/data/matrix_files/intrinsic_camera_properties.npz'
mtx, dist = load_intrinsic_camera_matrix(intrinsic_camera_matrix_file)

In [ ]:
# Baselink-camera matrix
baselink_camera_transformation_file = '/content/drive/My Drive/object_pose_estimation_online/data/matrix_files/mean_baselink_camera_transformation.npy'
bc_transform = get_numpy_data(baselink_camera_transformation_file)



> Als we nog even de volledige intrinsieke matrix bekijken: 

$$\begin{bmatrix} u'\\ v' \\ w' \end{bmatrix} = K *\begin{bmatrix} X\\ Y \\ Z \end{bmatrix}$$ 

> Dan zien we dat coördinaten $[X, Y, Z]$ worden omgezet in $[u', v', w']$. Om van pixelcoördinaten naar metrische coördinaten in camera frame te gaan moeten we deze matrix nog inverteren.

$$\begin{bmatrix} X\\ Y \\ Z \end{bmatrix} = K^{-1} *\begin{bmatrix} u'\\ v' \\ w' \end{bmatrix}$$ 






In [ ]:
# Invert matrix
mtx_inv = invert_transform(mtx)



> We zien ook dat het niet u en v is dat we nodig hebben voor een goede omvorming, maar u', v' en w'. We hebben gezien dat $u' = u * w'$  ,   $v' = v * w'$  ,  en $w' = Z$. Om van 2D terug naar 3D te kunnen gaan hebben we dus nog de diepte nodig van het punt. M.a.w de afstand van de pixel tot het centerpunt van het object. Dit kunnen we niet uit RGB beelden halen. De camera in het labo is echter een RGBD camera, die ook diepte kan meten. Hieruit hebben we gehaald dat het centerpunt van het object (de pixel die we berekend hebben) op 0.745 meter ligt van de pixel. Met deze info kunnen we dus $[u', v', w']$ bekomen. 



In [ ]:
# Convert (u, v) to (u', v', w') using Z
Z = 0.745
u_accent = mean_coordinate[0] * Z
v_accent = mean_coordinate[1] * Z
w_accent = Z



> Nu dat we het [u', v', w'] coördinaat hebben van het object, kunnen we de inverse van de intrinsieke matrix gebruiken om deze om te zetten naar een metrisch coördinaat in camera frame. 



In [ ]:
# Calculate position in camera frame
pixel_coordinate_accent = [u_accent, v_accent, w_accent]
c_p = np.dot(invert_transform(mtx), pixel_coordinate_accent)
c_p = np.append (c_p, [1]) # To make it a 4x1 vector
print("Coördinaat in camera frame: \n\n" + str(np.reshape(c_p, (4, 1))))



> Tot slot projecteren we dit nog naar baselink frame met de baselink-camera transformatie. 



In [ ]:
# Covert position to baselink frame
b_p = np.dot(bc_transform,  c_p)
print("Coördinaat in baselink frame: \n\n" + str(np.reshape(b_p, (4, 1))))



> Dit is dus het centerpunt van het object gedefiniëerd in het baselink frame van de robot. Om de robot naar een positie te kunnen sturen hebben we niet enkel een positie nodig maar hebben we een volledige transformatiematrix nodig van baselink naar target. Hieronder definiëren we nog welke oriëntatie we bij het object willen hebben door custom een 3x3 rotatiematrix te definiëren. Vervolgens brengen we die custom rotatiematrix en de berekende translatie matrix (positie) samen tot een volwaardige transformatiematrix. 



In [ ]:
# Create target orientation (custom)
R_target = np.array([[math.sqrt(2)/2, -math.sqrt(2)/2, 0],
       [-math.sqrt(2)/2, -math.sqrt(2)/2, 0],
       [0, 0, -1]])
print("Rotatie matrix: \n\n" + str(R_target))

# Create target position (calculated from image)
P_target = b_p
print("\nTranslatie matrix: \n\n" + str(np.reshape(P_target, (4, 1))))

# Create total transformation (R and T)
bt_transform = np.zeros((4, 4))
bt_transform[:3, :3] = R_target
bt_transform[:, 3] = P_target
print("\nTotale transformatiematrix: \n\n" + str(bt_transform))



> Dit is wat we wilden bekomen. We kunnen nu als aller laatste stap deze matrix opslaan en deze in het volgende script visualiseren en de robot ernaar toe laten bewegen. 



In [ ]:
# Save the baselink - target transformation matrix
baselink_target_transformation_file = '/content/drive/My Drive/object_pose_estimation_online/data/matrix_files/baselink_target_transformation'
save_to_numpy(baselink_target_transformation_file, bt_transform)

*  VRAAG 1: Wat is 'dilaten'? En wat is het nut ervan?
*  VRAAG 2: Wat is 'eroderen'? En wat is het nut ervan?
*  VRAAG 3: Wat is een 'contour'? En wat is het nut ervan?
*  VRAAG 4: Wat gebeurt er met de pixelcoördinaten van het punt als de Z-coördinaat groter wordt?
